<a href="https://colab.research.google.com/github/SATYAJEET-JADHAV/Github-final-project/blob/main/colab_notebooks/Part3_NLP_Sentiment_Analysis_for_Stocks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**TO** READ "Stock News Sentiment Analysis with Python": https://towardsdatascience.com/stock-news-sentiment-analysis-with-python-193d4b4378d4
https://finviz.com/quote.ashx?t=DAL&ty=c&ta=1&p=d

Additional idea: use the specific set of articles from the website - that may be available over a longer period of time, rather than NewsAPI


**Article outline**

1) **Intro**: 'Why news and NLP?'
The stock market can be influenced by the news: it can be any important thing about the company (eg new contract, new business line, strong manager hired , etc.), or  financial results (quarterly and annual earnings, profits , etc.) compared with analysts expectations .
In this article you will try to get automatically the list of news using the newsapi API, apply sentiment analysis, and compare the results with the stock/index prices (we covered the data piece for this in the prev email)

2) **News Api**: show the endpoints for get everything/ sources. Show how to get list of ids for only business English sources. Show the python wrapper and the functionality that is limited compared to the newsapi API (the idea is that you can get it with the requests library using the full functionality).
   It is not ideal though: you don't understand if the news are relevant to the stocks, you have only 1 month of history.
   Alternative to investigate:
   - Finviz https://finviz.com/quote.ashx?t=DAL&ty=c&ta=1&p=d
   - the article: https://towardsdatascience.com/stock-news-sentiment-analysis-with-python-193d4b4378d4
   You can have links of articles and their names that are highly relevant and hand-curated
  
3) **Sentiment analysis: introduce the library**
- https://medium.com/analytics-vidhya/simplifying-social-media-sentiment-analysis-using-vader-in-python-f9e6ec6fc52f
- the paper!!: http://comp.social.gatech.edu/papers/icwsm14.vader.hutto.pdf
  I like to cover not only compound, but also positive, neutral, and negative.
  We can also check what is more correlated with the graph movements.

  Show scores for several article names.

4)**The idea:** analyse one event with the news sentiment

Get one request for BRK.B (Berkshire Hathaway Inc.) stock around 9th July
- check the links list, descriptions, and short contents. Are the news similar, are they relevant to financials?
We selected 9th July and large capitalisation stock brk.b because there are many news about it and there was a jump in stock’s price. We will try to find the evidence why did that happen In the news coverage.

 What does it show on the news about brk.b stock. Can it be debugged ? (Show which words Influenced on positive/negative sentiment)
Compare the sentiment score vs. Stock rise and speculate what articles may influenced its growth

5) **Scale the analysis**: check daily news (about stocks) sentiment vs. growth of S&P500 index.

- 5.1) option 1 : all news from the en newspapers about stocks (we get max number =100 news via python library ).
- 5.2) option 2: we limit the sources only to 7 business related sources (it is 20-30 news about Stocks).

We compare both results vs. Index movement and find that business news sentiment is visually more correlated with the move .


In [1]:
# GLOBAL PARAMS - for one event

SHOW_ALL_ARTICLES = True
# SHOW_ALL_ARTICLES = False

#the day of the event
# STARTD = '22-Jun-2020'
# STARTD = '19-Jun-2020'
STARTD = '9-Jul-2020'


#the search phrase for News API
# KEYWRD = 'Inovio Pharmaceuticals stock'
# KEYWRD = 'Apple Inc.'
KEYWRD = 'Berkshire Hathaway Inc.'


#tiker
# tkr = "INO"
# tkr = 'AAPL'
tkr = 'BRK-B'
# tkr = 'LMND'


# 0) Imports

In [2]:
import sys
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
### Uncomment it when the script runs for the first time
nltk.download('vader_lexicon')
sia = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


In [3]:
!pip install newsapi-python

In [4]:
!pip install yfinance

In [5]:
from newsapi import NewsApiClient
#from newsapi.newsapi_client import NewsApiClient
from datetime import date, timedelta, datetime
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import yfinance as yf


In [6]:
# Show full output in Colab
# https://stackoverflow.com/questions/54692405/output-truncation-in-google-colab
pd.set_option('display.max_colwidth',1000)

# 1) News API BY KEYWORD - download some news on a search keyword for a specific date, sorted on relevancy for language =en, first 100 articles

## 1.0) Define functions calling news api

We use this function to call one end point to filter proper sources

In [7]:
# https://newsapi.org/docs/endpoints/sources
# https://github.com/mattlisiv/newsapi-python

def get_sources(category = None):
  newsapi = NewsApiClient(api_key='1900869fa01647fca0bdc19b4550daa0')
  sources = newsapi.get_sources()
  if category is not None:
    rez = [source['id'] for source in sources['sources'] if source['category'] == category and source['language'] == 'en']
  else:
    rez = [source['id'] for source in sources['sources'] if source['language'] == 'en']
  return rez


In [8]:
# 81 en sources
len(get_sources())

80

In [9]:
# 7 business sources
get_sources('business')

['australian-financial-review',
 'bloomberg',
 'business-insider',
 'financial-post',
 'fortune',
 'the-wall-street-journal']

In [10]:
type(date(2020,1,1))

datetime.date

In [11]:
# https://github.com/mattlisiv/newsapi-python/blob/master/newsapi/newsapi_client.py
#  https://newsapi.org/docs/endpoints/everything


def get_articles_sentiments(keywrd, startd, sources_list = None, show_all_articles = False):

  newsapi = NewsApiClient(api_key='1900869fa01647fca0bdc19b4550daa0')
  if type(startd) == str:
    my_date = datetime.strptime(startd,'%d-%b-%Y')
  else:
    my_date = startd
  # business_en_sources = get_sources('business','en')
  if sources_list:
    articles = newsapi.get_everything(q=keywrd,
                                      from_param = my_date.isoformat(),
                                      to = (my_date + timedelta(days = 1)).isoformat(),
                                      language="en",
                                      sources = ",".join(sources_list),
                                      sort_by="relevancy",
                                      page_size = 100)
  else:
     articles = newsapi.get_everything(q=keywrd,
                                      from_param = my_date.isoformat(),
                                      to = (my_date + timedelta(days = 1)).isoformat(),
                                      language="en",
                                      sort_by="relevancy",
                                      page_size = 100)
  article_content = ''

  date_sentiments = {}
  date_sentiments_list = []
  seen = set()

  for article in articles['articles']:
    if str(article['title']) in seen:
      continue
    else:
      seen.add(str(article['title']))
      article_content = str(article['title']) + '. ' + str(article['description'])

      sentiment = sia.polarity_scores(article_content)['compound']
      date_sentiments.setdefault(my_date, []).append(sentiment)
      date_sentiments_list.append((sentiment, article['url'],article['title'],article['description']))
  date_sentiments_l = sorted(date_sentiments_list, key=lambda tup: tup[0],reverse=True)
  sent_list = list(date_sentiments.values())[0]

  return pd.DataFrame(date_sentiments_list, columns=['Sentiment','URL','Title','Description'])

## 1.1) Test 2 versions for all news with q='stock': for ALL en sources

In [12]:
# Easy version when we don't filter the business source -- seems to be relevant though, but the description
# Get all sources in en
return_articles = get_articles_sentiments(keywrd= 'stock' ,startd = '21-Jul-2020',sources_list = None, show_all_articles= True)
return_articles.Sentiment.hist(bins=30,grid=False)
print(return_articles.Sentiment.mean())
print(return_articles.Sentiment.count())
print(return_articles.Description)



NewsAPIException: {'status': 'error', 'code': 'parameterInvalid', 'message': 'You are trying to request results too far in the past. Your plan permits you to request articles as far back as 2025-03-17, but you have requested 2020-07-21. You may need to upgrade to a paid plan.'}

Result: you can see 100 articles, with a lot of neutral sentiment,
and it is skewed towards very positive

In [ ]:
return_articles.sort_values(by='Sentiment', ascending=True)[['Sentiment','URL']].head(2)

In [ ]:
return_articles.sort_values(by='Sentiment', ascending=True)[['Sentiment','URL']].tail(2)

Top negative: https://www.reuters.com/article/india-nepal-palmoil-idUSL3N2ES1Y3
Nepal stops buying (New Dehli Suspended 39 oil import...)

In [ ]:
return_articles.sort_values(by='Sentiment', ascending=True)[['Sentiment','URL']].head(2)

From the article above: "TOKYO, July 21 (Reuters) - Japanese stocks rose on Tuesday as signs of progress in developing a COVID-19 vaccine boosted investor confidence in the outlook for future economic growth."

## 1.2 Same as 1.1, but try only business sources : different articles covered?

In [ ]:
# Look at only business sources in en
sources = get_sources('business')
return_articles = get_articles_sentiments('stock','21-Jul-2020',sources_list=sources, show_all_articles=True)
return_articles.Sentiment.hist(bins=30,grid=False)
print(return_articles.Sentiment.mean())
print(return_articles.Sentiment.count())
print(return_articles.Description)


In [ ]:
return_articles = get_articles_sentiments('stock','20-Jul-2020', sources_list=sources, show_all_articles=True)
return_articles.Sentiment.hist(bins=30,grid=False)
print(return_articles.Sentiment.mean())
print(return_articles.Sentiment.count())


In [ ]:
return_articles = get_articles_sentiments('stock','19-Jul-2020',show_all_articles=True)
return_articles.Sentiment.hist(bins=30,grid=False)
return_articles.Sentiment.mean()


# 2) Get sentiment for 1 month:  q= 'stocks', 'en' language, with and without business category filter

In [ ]:
from datetime import date

end_date = date.today()
start_date = date(year=end_date.year, month=end_date.month-1, day=end_date.day)

print('Start day = ', start_date)
print('End day = ', end_date)

current_day = start_date
business_sources = get_sources('business')
sentiment_all_score = []
sentiment_business_score = []
dates=[]


while current_day <= end_date:

  dates.append(current_day)

  sentiments_all = get_articles_sentiments(keywrd= 'stock' ,
                                          #  current_day.strftime('%b-%d-%Y'),
                                       startd = current_day,
                                       sources_list = None,
                                       show_all_articles= True)
  sentiment_all_score.append(sentiments_all.mean())

  sentiments_business = get_articles_sentiments(keywrd= 'stock' ,
                                      #  startd = current_day.strftime('%b-%d-%Y'),
                                       startd = current_day,
                                       sources_list = business_sources,
                                       show_all_articles= True)

  sentiment_business_score.append(sentiments_business.mean())

  current_day = current_day + timedelta(days=1)

In [ ]:
sentiments = pd.DataFrame([dates,np.array(sentiment_all_score),np.array(sentiment_business_score)]).transpose()

In [ ]:
sentiments.columns=['Date','All_sources_sentiment','Business_sources_sentiment']

In [ ]:
sentiments['Date'] = pd.to_datetime(sentiments['Date'])

In [ ]:
sentiments['All_sources_sentiment'] = sentiments['All_sources_sentiment'].astype(float)
sentiments['Business_sources_sentiment'] = sentiments['Business_sources_sentiment'].astype(float)

In [ ]:
sentiments.info()

In [ ]:
sentiments.set_index("Date", inplace=True)

In [ ]:
sentiments.head()

# 3) S&P 500 and Dow Jones index - reuse from the prev article

In [ ]:

# https://pydata.github.io/pandas-datareader/remote_data.html#stooq-index-data
# FRED: Federal Reserve Economic Data // fred,stlouisfed.org
import pandas_datareader.data as pdr
# from datetime import datetime
from datetime import date
end = date.today()
# start = datetime(2020,1,1)
start = datetime(year=end.year, month=end.month-1, day=end.day)
# end = datetime(2020,6,29)
print(f'Period 1 year until today: {start} to {end} ')

In [ ]:
spx_index = pdr.get_data_stooq('^SPX', start, end)
dji_index = pdr.get_data_stooq('^DJI',start,end)


In [ ]:
spx_index.index

In [ ]:
spx_index['Close'].plot(title='1 month price history for index S&P500 Index')

In [ ]:
dji_index['Close'].plot(title='1 month price history for index Dow Jones Index')

In [ ]:
sentiments_vs_snp = sentiments.join(spx_index['Close']).dropna()

# spx_index['Close'].join(sentiments).head()
  #  plot(title='1 month price history for index S&P500')

In [ ]:
sentiments_vs_snp.rename(columns={'Close':'s&p500_close'}, inplace=True)

In [ ]:
sentiments_vs_snp.head()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# https://stackoverflow.com/questions/31594549/how-do-i-change-the-figure-size-for-a-seaborn-plot
# https://stackoverflow.com/questions/47591650/second-y-axis-time-series-seaborn
sns.set(rc={'figure.figsize':(13.0,8.0)})


ax=sns.lineplot(data=sentiments_vs_snp['s&p500_close'], color="b",label='S&P500 Close price')
ax2 = plt.twinx()
sns.lineplot(data=sentiments_vs_snp["All_sources_sentiment"], color="g", ax=ax2, label='All sources sentiment')


In [ ]:
# BUSINESS SENTIMENT LOOKS CLOSER!

sns.set(rc={'figure.figsize':(13.0,8.0)})


ax=sns.lineplot(data=sentiments_vs_snp['s&p500_close'], color="b", label='S&P500 Close price')
ax2 = plt.twinx()
sns.lineplot(data=sentiments_vs_snp["Business_sources_sentiment"], color="g", ax=ax2, label='Business_sources_sentiment')

# 4.1) Individual cases - Shopify

In [ ]:
business_sources

In [ ]:
startd='29-Jul-2020'
shopify_stock = get_articles_sentiments("Shopify", startd, sources_list = business_sources, show_all_articles = True)

In [ ]:
shopify_stock

In [ ]:
shopify_stock_all = get_articles_sentiments("Shopify", startd, sources_list = None, show_all_articles = True)

In [ ]:
shopify_stock_all.Sentiment.mean()

In [ ]:
shopify_stock_all

In [ ]:
# Previous day
startd='28-Jul-2020'

shopify_stock_all_prevDay = get_articles_sentiments("Shopify", startd, sources_list = None, show_all_articles = True)

In [ ]:
shopify_stock_all_prevDay.Sentiment.mean()

In [ ]:
shopify_stock_all_prevDay

In [ ]:
from datetime import date

end_date = date.today()
start_date = date(year=end_date.year, month=end_date.month-1, day=end_date.day)

print('Start day = ', start_date)
print('End day = ', end_date)

current_day = start_date
sentiment_all_score = []
dates=[]


while current_day <= end_date:

  dates.append(current_day)

  sentiments_all = get_articles_sentiments(keywrd= 'Shopify' ,
                                          #  current_day.strftime('%b-%d-%Y'),
                                       startd = current_day,
                                       sources_list = None,
                                       show_all_articles= True)
  sentiment_all_score.append(sentiments_all.mean())

  current_day = current_day + timedelta(days=1)

In [ ]:
sentiments_shopify = pd.DataFrame([dates,np.array(sentiment_all_score)]).transpose()

In [ ]:
sentiments_shopify.columns

In [ ]:
# sentiments_shopify.columns=['Date','All_sources_sentiment']
# sentiments_shopify['Date'] = pd.to_datetime(sentiments_shopify['Date'])
# sentiments_shopify['All_sources_sentiment'] = sentiments_shopify['All_sources_sentiment'].astype(float)
sentiments_shopify.set_index("Date", inplace=True)

In [ ]:
sentiments_shopify.plot()

# 4.2) GAFA stock - 30 july earnings call :get news and sentiment for Google, Amazon, Facebook, Apple